### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Metabolic_Rate/GSE101492'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check the availability of gene expression data 
is_gene_available = True

# Identify the rows for variables 'Metabolic_Rate', 'age', 'gender'
# Based on the sample characteristics dictionary provided in the output from STEP 1

# Since 'Metabolic_Rate' is interpreted from 'insulin sensitivity', we use row 3
trait_row = 3

# Age is available in row 2
age_row = 2

# Gender is filled in row 1
gender_row = 1

# Define the conversion functions for each variable

def convert_trait(value):
    if 'insulin sensitive' in value:
        return 0  # Assuming 'insulin sensitive' means low metabolic rate
    elif 'insulin resistant' in value:
        return 1  # Assuming 'insulin resistant' means high metabolic rate
    return None

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(':')[-1].strip().lower()
    if gender == "female":
        return 0
    elif gender == "male":
        return 1
    return None

# Save cohort information
save_cohort_info('GSE101492', './preprocessed/Metabolic_Rate/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature data
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Metabolic_Rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Metabolic_Rate/trait_data/GSE101492.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'probeset_id'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
# Ensure the identifier key is present in gene_data
gene_data.index.name = 'ID'
gene_data = gene_data.reset_index()

# Apply gene mapping to convert from probe data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the converted gene_data
print(gene_data)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Metabolic_Rate/gene_data/GSE101492.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Metabolic_Rate')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE101492', './preprocessed/Metabolic_Rate/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Metabolic_Rate/GSE101492.csv'
    unbiased_merged_data.to_csv(csv_path)
